In [46]:
import pandas as pd
import re
import string
import time

from collections import Counter, defaultdict

from needleman_wunsch import needleman_wunsch

from fuzzywuzzy import fuzz

from operator import itemgetter

pd.set_option('display.max_rows', 1000)

In [2]:
with open('data/dataYiddish/hebis/json_parallels.txt') as f:
    text = f.read()

In [3]:
pairs = re.findall('\["(.+?)", "(.+?)"\]', text)

In [6]:
heb = []
lat = []

for pair in pairs:
    title = re.sub('[^\u0590-\u05fe \-]+', '', pair[1])
    heb.append(title)
    trans = re.sub('[^\w \-]+', '', pair[0])
    trans = re.sub('datl[\d\-]+', '', trans)
    lat.append(trans)

In [9]:
len(pairs)

6535

In [10]:
letterpairs = [(pair[0][0], pair[1][0].lower()) for pair in zip(heb, lat)]

In [ ]:
letterdict = defaultdict(str)

for tup in Counter(letterpairs).most_common()[::-1]:
    letterdict[tup[0][0]] = tup[0][1]

In [14]:
letterdict

defaultdict(str,
            {' ': ' ',
             'פ': 'f',
             'ג': 'g',
             'מ': 'm',
             'ל': 'l',
             'ח': 'ḥ',
             'כ': 'k',
             'ד': 'd',
             'ז': 'z',
             'ע': 'ʿ',
             'ת': 't',
             'א': 'a',
             'ה': 'h',
             'ש': 's',
             'ב': 'b',
             '-': '2',
             'ר': 'r',
             'ס': 's',
             'ט': 'ṭ',
             'צ': 't',
             'נ': 'n',
             'ק': 'k',
             'װ': 'ṿ',
             'י': 'y',
             '״': 's',
             'ו': 'ṿ',
             'ײ': 'y'})

In [105]:
letterdict['כ'] = 'kh'
letterdict['ך'] = 'kh'
letterdict['צ'] = 'ts'
letterdict['ץ'] = 'ts'
letterdict['ש'] = 'sh'
letterdict['-'] = '-'
letterdict['ע'] = 'e'
letterdict['ן'] = 'n'
letterdict['ם'] = 'm'
letterdict['ף'] = 'f'
#letterdict['א'] = 'a'
#letterdict['י'] = 'i'
#letterdict['ײ'] = 'ey'
#letterdict['־'] = '-'

for num in range(10):
    letterdict[str(num)] = str(num)
    
for punct in string.punctuation:
    letterdict[punct] = punct

In [110]:
wordlist = []
worddict = {'אין': 'in', 'און': 'un'}

for h, l in zip(heb, lat):
    matches = defaultdict(lambda: ('', 0),)
    for i, word in enumerate(h.split()):
        if word in worddict.keys():
            pseudotrans = worddict[word]
        else:
            pseudotrans = ''.join([letterdict[letter] for letter in word]) 
        ratios = []
        for j, trans in enumerate(l.split()):
            ratio = fuzz.ratio(pseudotrans.lower(), trans.lower()) - abs(i-j)
            ratios.append((trans, pseudotrans, ratio))
        maxratio = max(ratios, key=itemgetter(2))
        print(maxratio)
        if maxratio[2] >= 50:
            if matches[maxratio[0]][1] >= maxratio[2]:
                pass
            elif matches[maxratio[0]][1] < maxratio[2]:
                matches[maxratio[0]] = (word, maxratio[2])
    for key, val in matches.items():
        wordlist.append((val[0], key))
        print(val[0], key)
    print()

('Lomir', 'lamyr', 60)
('lernen', 'lernen', 100)
לאמיר Lomir
לערנען lernen

('lernbukh', 'lernbṿkh', 88)
('farn', 'farn', 100)
('driṭn', 'dryṭn', 80)
('yor', 'yar', 67)
('folḳshul', 'falkshṿl', 62)
לערנבוך lernbukh
פארן farn
דריטן driṭn
יאר yor
פאלקשול folḳshul

('tsuzamengeshṭelṭ', 'tsṿzamengeshṭelṭ', 94)
('durkh', 'dṿrkh', 80)
('di', 'dy', 50)
('lerer', 'lerer', 100)
('fun', 'fṿn', 67)
('der', 'der', 100)
('yidisher', 'ydysher', 80)
('shul', 'shṿl', 75)
('in', 'in', 100)
('Meḳsiḳe', 'meksyke', 57)
('loyt', 'lṭ', 33)
('Lebediḳe', 'lebedyke', 75)
('ḳlangen', 'klangen', 86)
('fun', 'fṿn', 67)
('Sh', 'sh', 100)
('Basṭomsḳi', 'basṭamsky', 67)
('un', 'un', 100)
('M', 'm', 100)
('Ḥayimson', 'ḥymsan', 71)
צוזאמענגעשטעלט tsuzamengeshṭelṭ
דורך durkh
די di
לערער lerer
פון fun
דער der
ײדישער yidisher
שול shul
אין in
מעקסיקע Meḳsiḳe
לעבעדיקע Lebediḳe
קלאנגען ḳlangen
ש Sh
באסטאמסקי Basṭomsḳi
און un
מ M
חײמסאן Ḥayimson

('Bastomski', 'basṭamsky', 67)
('Shelomoh', 'shlmh-', 71)
באסטאמסקי Bastomski
ש

סלאוועס Slowes

('Slovès', 'slṿbs', 55)
('Henri', 'ḥyym-', 0)
סלובס Slovès

('Nju-Jork', 'nyṿ', 18)
('Nju-Jork', 'yark', 32)

('IKUF', 'aykṿf', 44)

('Zeks', 'zeks', 100)
('jidiše', 'yydyshe', 31)
('kriṭiker', 'kryṭyker', 75)
זעקס Zeks
קריטיקער kriṭiker

('Aleksander', 'aleksander', 100)
('Tzederboim', 'tsederbṿym', 70)
('J', 'yy', 0)
('Lerner', 'lerner', 99)
('Ḥ', 'yḥ', 65)
('Rawnitzki', 'raṿṿnytsky', 51)
('Aleksander', 'yṿal', 23)
('Enṭin', 'enṭyn', 78)
('Joiel', 'bel', 50)
('Bal-Maḥšoves', 'mḥshbṿt', 31)
('Bal-Maḥšoves', 'sh', 14)
('Niger', 'nyger', 79)
אלעקסאנדער Aleksander
צעדערבוים Tzederboim
לערנער Lerner
יח Ḥ
ראווניצקי Rawnitzki
ענטין Enṭin
בעל Joiel
ניגער Niger

('N', 'nḥṿm', 40)
('B', 'brṿkh', 33)
('Minkow', 'mynkaṿṿ', 46)

('Minkoff', 'mynkṿb', 46)
('Nahum', 'nḥṿm', 44)
('B', 'brṿkh-', 29)

('Buenos-Aires', 'bṿenas', 56)
('Buenos-Aires', 'ayyres', 43)
בוענאס Buenos-Aires

('Farlag', 'farlag', 100)
('Jidbuch', 'yydbṿkh', 43)
פארלאג Farlag

('Sotsyale', 'satsyale', 88)
('refor

('firer', 'fyrer', 80)
('fun', 'fṿn', 67)
('der', 'der', 100)
('ṿelṭ', 'ṿṿelṭ', 89)
געשטאלטן geshṭalṭn
און un
בילדער bilder
פון fun
די di
פירער firer
דער der
וועלט ṿelṭ

('Tsevi', 'tsby', 44)
('Kahn', 'khhn', 75)
כהן Kahn

('Di', 'dy', 50)
('heldn', 'heldn', 100)
('in', 'in', 100)
('der', 'der', 100)
('ṿelṭ-geshikhṭe', 'ṿṿelṭ-geshykhṭe', 90)
('geshṭalṭn', 'geshṭalṭn', 100)
('un', 'un', 100)
('bilder', 'bylder', 83)
('fun', 'fṿn', 67)
('di', 'dy', 50)
('firer', 'fyrer', 80)
('fun', 'fṿn', 67)
('der', 'der', 100)
('ṿelṭ', 'ṿṿelṭ', 89)
('Tsevi', 'tsby', 44)
('Kahn', 'khhn', 75)
די Di
העלדן heldn
אין in
דער der
וועלט-געשיכטע ṿelṭ-geshikhṭe
געשטאלטן geshṭalṭn
און un
בילדער bilder
פון fun
די di
פירער firer
וועלט ṿelṭ
כהן Kahn

('Di', 'dy', 50)
('heldn', 'heldn', 100)
('in', 'in', 100)
('der', 'der', 100)
('ṿelṭ-geshikhṭe', 'ṿṿelṭ-geshykhṭe', 90)
די Di
העלדן heldn
אין in
דער der
וועלט-געשיכטע ṿelṭ-geshikhṭe

('geshṭalṭn', 'geshṭalṭn', 100)
('un', 'un', 100)
('bilder', 'bylder', 83)
('fun', 'f

('Unzer', 'aṿnzer', 73)
('Tsayṭ', 'tsyyṭ', 80)
אונזער Unzer
צייט Tsayṭ

('Doyres', 'dṿrṿt', 36)
('Bundisṭn', 'bṿndysṭn', 75)
בונדיסטן Bundisṭn

('redagirṭ', 'redagyrṭ', 88)
('fun', 'fṿn', 67)
('Y', 'y', 100)
('Sh', 'sh', 100)
('Herts', 'herts', 100)
רעדאגירט redagirṭ
פון fun
י Y
ש Sh
הערץ Herts

('Doyres', 'dṿrṿt', 36)
('Bundisṭn', 'bṿndysṭn', 75)
('redagirṭ', 'redagyrṭ', 88)
('fun', 'fṿn', 67)
('Y', 'y', 100)
('Sh', 'sh', 100)
('Herts', 'herts', 100)
בונדיסטן Bundisṭn
רעדאגירט redagirṭ
פון fun
י Y
ש Sh
הערץ Herts

('Doyres', 'dṿrṿt', 36)
('Bundisṭn', 'bṿndysṭn', 75)
בונדיסטן Bundisṭn

('redagirṭ', 'redagyrṭ', 88)
('fun', 'fṿn', 67)
('Y', 'y', 100)
('Sh', 'sh', 100)
('Herts', 'herts', 100)
רעדאגירט redagirṭ
פון fun
י Y
ש Sh
הערץ Herts

('Hertz', 'herts', 80)
('Jacob', 'yekb', 22)
('S', 'shlṿm-', 29)
הערץ Hertz

('Nyu', 'nyṿ', 67)
('Yorḳ', 'yark', 50)
ניו Nyu
יארק Yorḳ

('Unzer', 'aṿnzer', 73)
('Tsayṭ', 'tsyyṭ', 80)
אונזער Unzer
צייט Tsayṭ

('Di', 'dy', 50)
('geshikhṭe', 'geshykhṭe', 89

וועגנ ṿegn

('lider', 'lyder', 80)
לידער lider

('Sh', 'sh', 100)
('Halḳin', 'ḥlkyn', 36)
ש Sh

('Halḳin', 'halkyn', 67)
('Shemuʾel', 'shmṿel-', 67)
האלקין Halḳin
שמועל- Shemuʾel

('Di', 'dy', 50)
('lange', 'lange', 100)
('nachṭ', 'nakhṭ', 80)
די Di
לאנגע lange
נאכט nachṭ

('Eliezer', 'alyezr', 62)
('Grinberg', 'grynberg', 88)
אליעזר Eliezer
גרינבערג Grinberg

('Greenberg', 'grynbrg', 75)
('Eliezer', 'alyezr-', 57)
גרינברג Greenberg
אליעזר- Eliezer

('Nju-Jork', 'nyṿ-yark', 50)
ניו-יארק Nju-Jork

('Lipkaner', 'lyfkaner', 75)
('Besaraber', 'besaraber', 100)
('Sosaieti', 'sasyyeṭy', 38)
ליפקאנער Lipkaner
בעסאראבער Besaraber

('In', 'in', 100)
('fremdn', 'fremdn', 100)
('land', 'land', 100)
אין In
פרעמדן fremdn
לאנד land

('M', 'm', 100)
('Olitzki', 'alytsky', 43)
מ M

('Farṿoḳsene', 'farṿṿaksene', 76)
('ṿegn', 'ṿṿegn', 89)
פארוואקסענע Farṿoḳsene
וועגן ṿegn

('lider', 'lyder', 80)
('un', 'un', 100)
('poemes', 'faemes', 67)
לידער lider
און un
פאעמעס poemes

('Ḥayim', 'ḥyym', 67)
('Grade', 

...

יידישער yidisher
קולטור־קאָנגעס ḳulṭur-ḳongres

('Di', 'dy', 50)
('geshikhṭe', 'geshykhṭe', 89)
('fun', 'fṿn', 67)
('der', 'der', 100)
('liṭeraṭur', 'lyṭeraṭṿr', 78)
('bay', 'by', 80)
('Yidn', 'ydn', 86)
די Di
געשיכטע geshikhṭe
פון fun
דער der
ליטעראטור liṭeraṭur
בײ bay
ײדן Yidn

('Yiśraʾel', 'yshral', 57)
('Tsinberg', 'tsynbrg', 80)
ישראל Yiśraʾel
צינברג Tsinberg

('Di', 'dy', 50)
('geshikhṭe', 'geshykhṭe', 89)
('fun', 'fṿn', 67)
('der', 'der', 100)
('liṭeraṭur', 'lyṭeraṭṿr', 78)
('bay', 'by', 80)
('Yidn', 'ydn', 86)
('Yiśroʾel', 'yshral', 43)
('Tsinberg', 'tsynbrg', 80)
די Di
געשיכטע geshikhṭe
פון fun
דער der
ליטעראטור liṭeraṭur
בײ bay
ײדן Yidn
צינברג Tsinberg

('Berliner', 'berlynger', 82)
('haśkoleh', 'hshlkhh', 40)
בערלינגער Berliner

('Buenos-Aires', 'bṿenas', 56)
('Buenos-Aires', 'ayyres', 43)
בוענאָס Buenos-Aires

('Alṿelṭlekher', 'alṿṿelṭlekher', 96)
('yidisher', 'yydysher', 75)
('ḳulṭur-ḳongres', 'kṿlṭṿrkanges', 54)
אַלוועלטלעכער Alṿelṭlekher
יידישער yidisher
קולטור־קאָנגעס ḳ

פארן farn
פאלק folḳ

('Erets', 'arts', 67)
('Yiśraʾel', 'yshral', 57)
ארץ Erets
ישראל Yiśraʾel

('Lider', 'lyder', 80)
('fun', 'fṿn', 67)
('ḥurbn', 'ḥṿrbn', 80)
לידער Lider
פון fun
חורבן ḥurbn

('700-705', 'tsshtshsh', 0)

('tsuzamengeshṭelṭ', 'tsṿzamengeshṭel', 90)
('Ḳadye', 'kadye', 80)
('Molodoṿsḳi', 'maladaṿṿsky', 48)
צוזאמענגעשטעל tsuzamengeshṭelṭ
קאדיע Ḳadye

('Molodowsky', 'mṿlṿdṿbsky', 60)
('Kadia', 'kdyh-', 40)
מולודובסקי Molodowsky

('ha-Shir', 'h', 25)
('ha-Shir', 'shyr', 54)
('ṿe-ha-shevaḥ', 'ṿhshbḥ', 55)
שיר ha-Shir
והשבח ṿe-ha-shevaḥ

('leḳeṭ', 'lkṭ', 50)
('shirim', 'shyrym', 67)
('be-ʿIdish', 'baydysh', 50)
לקט leḳeṭ
שירים shirim
באידיש be-ʿIdish

('Bene', 'bny', 57)
('Beraḳ', 'brk', 50)
בני Bene
ברק Beraḳ

('Yahlam', 'yhlm', 80)
יהלם Yahlam

('Anṭologye', 'anṭalagye', 78)
('fun', 'fṿn', 67)
('der', 'der', 100)
('yidisher', 'yydysher', 75)
('liṭeraṭur', 'lyṭeraṭṿr', 78)
('liṭeraṭur', 'far', 32)
('Anṭologye', 'yṿgnṭ', 23)
('yidisher', 'tsṿṿyyṭer', 31)
('Anṭologye', 'ṭyyl'

('redagirt', 'redagyrṭ', 75)
('fun', 'fṿn', 67)
('M', 'm', 100)
('Speḳṭor', 'sfekṭar', 57)
רעדאגירט redagirt
פון fun
מ M
ספעקטאר Speḳṭor

('2-ṭes', '-ṭes', 89)
('bukh', 'bṿkh', 75)
-טעס 2-ṭes
בוך bukh

('Hoyz-fraynd', 'hzfrynd', 78)
('a', 'a', 100)
('hisṭorish-liṭerarisches', 'hysṭaryshlyṭeraryshes', 73)
('bukh', 'bṿkh', 75)
('redagirt', 'redagyrṭ', 75)
('fun', 'fṿn', 67)
('M', 'm', 100)
('Spektor', 'sfekṭar', 57)
הױז־פרײנד Hoyz-fraynd
א a
היסטאריש־ליטערארישעס hisṭorish-liṭerarisches
בוך bukh
רעדאגירט redagirt
פון fun
מ M
ספעקטאר Spektor

('Speḳṭor', 'sfekṭar', 57)
('Mordekhai', 'mmrdkhy-', 59)
ספעקטאר Speḳṭor
ממרדכי- Mordekhai

('Hoyz-fraynd', 'hzfrynd', 78)
הױז־פרײנד Hoyz-fraynd

('a', 'a', 100)
('hisṭorish-liṭerarisches', 'hysṭaryshlyṭeraryshes', 73)
('bukh', 'bṿkh', 75)
א a
היסטאריש־ליטערארישעס hisṭorish-liṭerarisches
בוך bukh

('redagirt', 'redagyrṭ', 75)
('fun', 'fṿn', 67)
('M', 'm', 100)
('Speḳṭor', 'sfekṭar', 57)
רעדאגירט redagirt
פון fun
מ M
ספעקטאר Speḳṭor

('1-ṭes', '-ṭes', 

('oyf', 'aṿyf', 56)
('der', 'der', 99)
('Yidishe', 'yydysher', 66)
('bine', 'byne', 74)
ארויסגעגעבן Aroysgegebn
דורך durkh
דעם dem
געזעלשאפטלעכן Gezelshafṭlekhn
קאמיטעט ḳomiṭeṭ
צו tsu
פייערן fayern
יאר yor
טעטיקייט ṭeṭiḳeyṭ
פון fun
שמעון Shimʿon
דזשיגאן Dzshigan
אויף oyf
דער der
יידישער Yidishe
בינע bine

('Tel-Aviv', 'tl', 40)
('Tel-Aviv', 'abyb', 16)

('Defus', 'dfṿs', 67)
('Or-li', 'aṿrly', 40)
דפוס Defus

('Zikhroynes', 'zkhrṿnṿt', 56)
('fun', 'fṿn', 67)
('a', 'a', 100)
('maran', 'maran', 100)
('in', 'in', 100)
('der', 'der', 100)
('teḳufeh', 'tkṿfh', 50)
('fun', 'fṿn', 67)
('der', 'der', 100)
('natsi-ḳaṭasṭrofe', 'natsy-kaṭasṭrafe', 81)
זכרונות Zikhroynes
פון fun
א a
מאראן maran
אין in
דער der
תקופה teḳufeh
נאצי-קאטאסטראפע natsi-ḳaṭasṭrofe

('Zikhronot', 'zkhrṿnṿt', 71)
('shel', 'shl', 86)
('anus', 'anṿs', 75)
('bi-teḳufat', 'btkṿft', 50)
('ha-Shoʾah', 'hshṿah', 67)
('ha-natsit', 'hnatsyt', 75)
זכרונות Zikhronot
של shel
אנוס anus
בתקופת bi-teḳufat
השואה ha-Shoʾah
הנאצית ha-natsit


פארווערטס Forṿerṭs

('Di', 'dy', 50)
('geshikhṭe', 'geshykhṭe', 89)
('fun', 'fṿn', 67)
('idishen', 'aydyshen', 67)
('ṭheaṭer', 'ṭheaṭer', 100)
די Di
געשיכטע geshikhṭe
פון fun
אידישען idishen
טהעאטער ṭheaṭer

('tsṿey', 'tsṿṿyy', 73)
('ṭoyzenṭ', 'ṭṿyzenṭ', 86)
('yohr', 'yahr', 75)
('ṭheaṭer', 'ṭheaṭer', 100)
('bay', 'byy', 67)
('iden', 'ayden', 67)
צוויי tsṿey
טויזענט ṭoyzenṭ
יאהר yohr
טהעאטער ṭheaṭer
ביי bay
אידען iden

('B', 'b', 100)
('Gorin', 'garyn', 60)
('Y', 'y', 100)
('Goido', 'gṿyda', 40)
ב B
גארין Gorin
י Y

('Y', 'dy', 54)
('geshikhṭe', 'geshykhṭe', 89)
('fun', 'fṿn', 67)
('idishen', 'aydyshen', 67)
('ṭheaṭer', 'ṭheaṭer', 100)
('tsṿey', 'tsṿṿyy', 73)
('ṭoyzenṭ', 'ṭṿyzenṭ', 86)
('yohr', 'yahr', 75)
('ṭheaṭer', 'ṭheaṭer', 100)
('bay', 'byy', 67)
('iden', 'ayden', 67)
('B', 'b', 100)
('Gorin', 'garyn', 60)
('Y', 'y', 100)
('Goido', 'gṿyda', 40)
י Y
געשיכטע geshikhṭe
פון fun
אידישען idishen
טהעאטער ṭheaṭer
צוויי tsṿey
טויזענט ṭoyzenṭ
יאהר yohr
ביי bay
אידען iden
ב B
גארין Gorin

(

In [111]:
len(wordlist)

19052

In [112]:
unique_wordlist = list(set(wordlist))
len(unique_wordlist)

5322

In [ ]:
pair_df = pd.DataFrame({'yiddish': [tup[0] for tup in wordlist], 'translit': [tup[1] for tup in wordlist]})

In [114]:
pair_df.groupby(['yiddish', 'translit']).size().head(1000)

yiddish                  translit                
-                        -                            26
                         1-2                           1
-טן                      100-ṭen                       1
                         5-ṭn                          2
                         6-ṭn                          2
-טעס                     1-ṭes                         1
                         2-ṭes                         1
                         3-ṭes                         1
                         4-ṭes                         1
-טער                     50-ṭer                        1
-סטער                    25-sṭer                       1
ֻנטרגַנג                 unṭergang                     2
ֻרַלטר                   uralṭer                       1
א                        A                            84
                         AB                            1
                         a                           107
                         ʾA           

In [118]:
pair_df['source'] = 'hebis'

In [119]:
pair_df.drop_duplicates().to_csv('data/processed-parallel/hebis.csv', index=False)